In [119]:
import csv,sys
import math,random
import subprocess
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve

In [120]:
data = pd.read_csv("DSL-StrongPasswordData.csv")

In [121]:
subjects = data["subject"].unique()

In [122]:
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer

In [123]:
class Point:
    #An point in n dimensional space
    def __init__(self, coords):
    #coords - A list of values, one per dimension
        self.coords = coords
        self.n = len(coords)

    def __repr__(self):
        return str(self.coords)

In [124]:
class Cluster:
    #A set of points and their centroid

    def __init__(self, points):
    #points - A list of point objects

        if len(points) == 0: 
            raise Exception("ILLEGAL: empty cluster")
        # The points that belong to this cluster
        self.points = points

        # The dimensionality of the points in this cluster
        self.n = points[0].n
    
        # Assert that all points are of the same dimensionality
        for p in points:
            if p.n != self.n: raise Exception("ILLEGAL: wrong dimensions")
    
        # Set up the initial centroid (this is usually based off one point)
        self.centroid = self.calculateCentroid()

    def __repr__(self):
    #String representation of this object
        return str(self.points)

    def update(self, points):
    #Returns the distance between the previous centroid and the new after
    #recalculating and storing the new centroid.
        old_centroid = self.centroid
        self.points = points
        self.centroid = self.calculateCentroid()
        shift = getDistance(old_centroid, self.centroid) 
        return shift

    def calculateCentroid(self):
    #Finds a virtual center point for a group of n-dimensional points
        numPoints = len(self.points)
        # Get a list of all coordinates in this cluster
        coords = [p.coords for p in self.points]
        # Reformat that so all x's are together, all y'z etc.
        unzipped = zip(*coords)
        # Calculate the mean for each dimension
        centroid_coords = [math.fsum(dList)/numPoints for dList in unzipped]

        return Point(centroid_coords)


In [125]:
def getDistance(a, b):

#Euclidean distance between two n-dimensional points.
#Note: This can be very slow and does not scale well
    #if a.n != b.n:
    #    raise Exception("ILLEGAL: non comparable points")

    ret = reduce(lambda x,y: x + pow((a.coords[y]-b.coords[y]), 2),range(a.n),0.0)
    return math.sqrt(ret)

In [126]:
def testing(clusters,test_genuine,test_imposter):
    user_scores = []
    imposter_scores = []
    for i in range(len(test_genuine)):
        min_distance = 1e+10
        for j in range(len(clusters)):
            cur_score = getDistance(test_genuine[i],clusters[j].centroid)
            if cur_score < min_distance:
                min_distance = cur_score
        user_scores.append(min_distance)
            
    for i in range(len(test_imposter)):
        min_distance = 1e+10
        for j in range(len(clusters)):
            cur_score = getDistance(test_imposter[i],clusters[j].centroid)
            if cur_score < min_distance:
                min_distance = cur_score
        imposter_scores.append(min_distance)
    return user_scores,imposter_scores

In [127]:
def kmeans(points, k, cutoff):

    # Pick out k random points to use as our initial centroids
    initial = random.sample(points, k)

    # Create k clusters using those centroids
    clusters = [Cluster([p]) for p in initial]

    # Loop through the dataset until the clusters stabilize
    loopCounter = 0
    while True:
        # Create a list of lists to hold the points in each cluster
        lists = [ [] for c in clusters]
        clusterCount = len(clusters)

        # Start counting loops
        loopCounter += 1
        # For every point in the dataset ...
        for p in points:
            # Get the distance between that point and the centroid of the first
            # cluster.
            smallest_distance = getDistance(p, clusters[0].centroid)

            # Set the cluster this point belongs to
            clusterIndex = 0

            # For the remainder of the clusters ...
            for i in range(clusterCount - 1):
                # calculate the distance of that point to each other cluster's
                # centroid.
                distance = getDistance(p, clusters[i+1].centroid)
                # If it's closer to that cluster's centroid update what we
                # think the smallest distance is, and set the point to belong
                # to that cluster
                if distance < smallest_distance:
                    smallest_distance = distance
                    clusterIndex = i+1
            lists[clusterIndex].append(p)

        # Set our biggest_shift to zero for this iteration
        biggest_shift = 0.0

        # As many times as there are clusters ...
        for i in range(clusterCount):
            # Calculate how far the centroid moved in this iteration
            shift = clusters[i].update(lists[i])
            # Keep track of the largest move from all cluster centroid updates
            biggest_shift = max(biggest_shift, shift)

        # If the centroids have stopped moving much, say we're done!
        if biggest_shift < cutoff:
            print "Converged after %s iterations" % loopCounter
            break
    return clusters

In [128]:
def evaluate():
        eers = []
        k = 3
        cut_off = 0.5
        for subject in subjects:
            
            user_scores = []
            imposter_scores = []
    
            # Consider current subject as genuine and rest as imposters
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            train = genuine_user_data[:200]
            train = train.values
            train = np.array(train)
            train = list(train)
            temp = []
            for p in train:
                temp.append(list(p))
            train = temp
            points = []
            for p in train:
                points.append(Point([float(elements) for elements in p]))
            temp = []
            # True set (200 records)
            test_genuine = genuine_user_data[200:]
            test_genuine = test_genuine.values
            test_genuine = np.array(test_genuine)
            test_genuine = list(test_genuine)
            for p in test_genuine:
                temp.append(list(p))
            test_genuine = temp
            # False set (250 records, 5 per imposter, 50 imposters in all)
            temp = []
            test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            test_imposter = test_imposter.values
            test_imposter = np.array(test_imposter)
            test_imposter = list(test_imposter)
            for p in test_imposter:
                temp.append(list(p))
            test_imposter = temp
            points_test_genuine = []
            points_test_imposter = []
            for p in test_genuine:
                points_test_genuine.append(Point([float(elements) for elements in p]))
            for p in test_imposter:
                points_test_imposter.append(Point([float(elements) for elements in p]))
            clusters = kmeans(points,k,cut_off)
            
            user_scores,imposter_scores = testing(clusters,points_test_genuine,points_test_imposter)
    
            eers.append(evaluateEER(user_scores, imposter_scores))
        
        return np.mean(eers), np.std(eers)

In [129]:
evaluate()

Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 2 iterations
Converged after 2 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 3 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 1 iterations
Converged after 2 iterations
Converged after 4 iterations
Converged after 2 iterations
Converged after 2 iterations
Converged after 1 iterations
Converged afte

(0.15594482649747973, 0.072058023537407423)